In [1]:
%load_ext autoreload
%autoreload 2
%load_ext snakeviz
%load_ext line_profiler


from coeditor.common import *
import os

os.chdir(proj_root())

In [2]:
from coeditor.git import get_commit_history, CommitInfo

repo_root = proj_root()
# repo_root = get_dataset_dir("tiny") / "repos/test/gym"

commits = get_commit_history(repo_root, 5)
# commits = [CommitInfo("9dea81b48a2e1d8f7e7a81211c0f09f627ee61a9", (), "omit"),
#            CommitInfo("6a4cefd875f6547fc0bb43e02b14bb21d70fda9a", (), "omit")]
for c in commits[:3]:
    print(c)


CommitInfo(hash='84cfd5206348ecc3f54d202b830f803d8a03f26f', parents=('a1e2b73ab836924d0b1f9ed88e4fd90e7a6f61e6',), msg='Add ablation: dense attention.')
CommitInfo(hash='a1e2b73ab836924d0b1f9ed88e4fd90e7a6f61e6', parents=('ecdbdc3875e47887ff3c0320fd1367af28d0a491',), msg='Implement ablation: current_code_only.')
CommitInfo(hash='ecdbdc3875e47887ff3c0320fd1367af28d0a491', parents=('ad918b35e2b8314f30a7f8ffc1e957c9f49956df',), msg='Exclude builtins defs in ctx by default.')


In [3]:
from coeditor.scoped_changes import edits_from_commit_history, _tlogger
from coeditor.c3problem import C3ProblemGenerator, C3ProblemTokenizer, JediUsageAnalyzer
from coeditor.experiments.code_completion import FIMProblem, C3CompletionGenerator

# generator = C3ProblemGenerator(JediUsageAnalyzer(include_builtins=True))
generator = C3CompletionGenerator()

workdir = proj_root() / "../temp-1"
# subprocess.run(["rm", "-r", workdir])
_tlogger.clear()
problems = edits_from_commit_history(
    repo_root, commits, workdir, change_processor=generator
)
print(f"{len(problems) = }")
# for err, count in generator.analyzer.error_counts.items():
#     print(f"({count=}): {err}")
display(_tlogger.as_dataframe())


processing commits: 100%|██████████| 4/4 [00:04<00:00,  1.13s/it]

len(problems) = 13


,name,count,avg_time,total_time
5,process_change,4,0.663314,2.653256
1,parse_module,42,0.021610,0.907621
0,checkout,13,0.008255,0.107321
2,JModuleChange.from_modules,8,0.013032,0.104256
4,pre_edit_analysis,4,0.000002,0.000007
3,post_edit_analysis,4,0.000001,0.000005


In [4]:
from coeditor.encoding import decode_tokens

ex_id = 8
input, output = problems[ex_id].to_codet5_format()
print_sections(
    ("output", decode_tokens(output)),
    ("input", decode_tokens(input)),
)

--------------------------------------------------------------------------------
output:
            save_steps=max(500, min(10000, epoch_steps // 5)),
--------------------------------------------------------------------------------
input:
<s>    out_tks: TokenSeq
    score: float
    n_samples: int

class RetrievalModelPrediction(TypedDict):
    input_ids: TokenSeq
    output_ids: TokenSeq
    labels: TokenSeq
    references: list[TokenSeq]

class AttentionMode(enum.Enum):
    basic = enum.auto()
    query2ref = enum.auto()
    bidirectional = enum.auto()

class RetrievalEditorModel(T5PreTrainedModel):
    is_parallelizable = False
    supports_gradient_checkpointing = False

    """
    A CodeT5 model that takes in multiple reference code snippets and a
    query code snippet with multiple masked spans and perdicts the maksed spans.

    While the computational cost of a normal CodeT5 encoder increases quadratically,
    this model only increases linearly with the number of reference

In [4]:
from coeditor.encoding import encode_diffs, _BaseTokenizer, Add_id, decode_tokens
from coeditor.c3problem import C3ProblemSimpleSplit, C3ProblemChangeInlining, C3ToCodeCompletion

prob_trans = C3ProblemChangeInlining()
# %lprun -T output/lprof.txt -f C3ProblemTokenizer.tokenize_problem edits = [e for p in problems for e in encoder.tokenize_problem(p)]
new_probs = [p1 for p in problems for p1 in prob_trans.transform(p)]


In [27]:
ex_id = 3
encoder = C3ProblemTokenizer(current_code_only=False)
print("delta: " + str(new_probs[ex_id].span.delta))
print("edit range: " + str(new_probs[ex_id].edit_line_ids))
print("transformations: " + str(new_probs[ex_id].transformations))
edits = [encoder.tokenize_problem(p) for p in new_probs]
print(edits[ex_id].show())

delta: TkDelta(
  2: ('<add>             len_in = rand.randint(64, 512)', '<add>             len_out = rand.randint(14, 256)', '<add>             input_ids = 5 * torch.ones(1, len_in, dtype=torch.long, device=self.device)', '<del>')
  3: ('<del>',)
  4: ('<del>',)
  7: ('<add>             labels = 5 * torch.ones(1, len_out, dtype=torch.long, device=self.device)', '<del>')
)
edit range: range(0, 11)
transformations: ('split',)
--------------------------------------------------------------------------------
path: coeditor.model/RetrievalEditorModel.profile_run
n_references: 9
total_reference_tks: 2957
project: temp-1
commit: CommitInfo(hash='31e1ac706f81baaf059aaee0787edab67bda6f3b', parents=('105bd32bd2520f951e8003a3e08c82d784029b2d',), msg='Improve C3DataLoader performance. - move _post_process into pmap. - support `disable_unchanged_refs`.')
========Ground Truth========
 <2>:<add>             len_in = rand.randint(64, 512)
     <add>             len_out = rand.randint(14, 256)
     <a

In [28]:
from coeditor.c3problem import _problem_to_current

encoder = C3ProblemTokenizer(current_code_only=True)
cprob = _problem_to_current(new_probs[ex_id])
print("delta: " + str(cprob.span.delta))
print("edit range: " + str(cprob.edit_line_ids))
print("transformations: " + str(cprob.transformations))
print(encoder.tokenize_problem(new_probs[ex_id]).show())

delta: TkDelta(
  2: ('<add>             len_in = rand.randint(64, 512)', '<add>             len_out = rand.randint(14, 256)', '<add>             input_ids = 5 * torch.ones(1, len_in, dtype=torch.long, device=self.device)', '<del>')
  3: ('<del>',)
  4: ('<del>',)
  7: ('<add>             labels = 5 * torch.ones(1, len_out, dtype=torch.long, device=self.device)', '<del>')
)
edit range: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
transformations: ('split',)
--------------------------------------------------------------------------------
path: coeditor.model/RetrievalEditorModel.profile_run
n_references: 7
total_reference_tks: 2585
project: temp-1
commit: CommitInfo(hash='31e1ac706f81baaf059aaee0787edab67bda6f3b', parents=('105bd32bd2520f951e8003a3e08c82d784029b2d',), msg='Improve C3DataLoader performance. - move _post_process into pmap. - support `disable_unchanged_refs`.')
========Ground Truth========
 <2>:<add>             len_in = rand.randint(64, 512)
     <add>             len_out = rand.ra

In [6]:
%%timeit -n 1 -r 2
analysis = UsageAnalysis(pyproj, add_implicit_rel_imports=True, record_type_usages=True)

9.82 s ± 26 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)


In [34]:
%%timeit -n 1 -r 2
PythonProject.from_root(proj_root())

4.79 s ± 21.6 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
